<a href="https://colab.research.google.com/github/Parthi1212-dotcom/Investment-Portfolio-through-Evolutionary-algorithms/blob/main/Merging_csv_files_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def calculate_stock_fitness(filepath):
    """
    Loads stock data from a CSV file and calculates a custom fitness score.

    The fitness score is based on the average of several rank columns from the dataset.
    A lower score indicates a "fitter" stock, as it implies better average ranks.

    Args:
        filepath (str): The path to the CSV file containing the stock data.

    Returns:
        pandas.Series: A Series containing the fitness score for each stock ticker,
                       sorted to show the fittest stocks first (lowest score).
                       Returns None if the file cannot be found or an error occurs.
    """
    try:
        # Load the dataset from the specified file path into a pandas DataFrame.
        df = pd.read_csv(filepath)
        print("Successfully loaded the dataset.")
    except FileNotFoundError:
        # Handle the case where the file does not exist at the given path.
        print(f"Error: The file '{filepath}' was not found.")
        return None
    except Exception as e:
        # Handle other potential errors during file loading.
        print(f"An error occurred while loading the file: {e}")
        return None

    # Define the columns that we will use to calculate our fitness score.
    # These columns represent different performance ranks.
    rank_columns = ['McapRank', 'TurnRank', 'VolatilityRank', 'PriceRank']

    # --- Data Cleaning and Preparation ---
    print("Cleaning and preparing data...")
    for col in rank_columns:
        # Ensure the rank columns are treated as numbers. If a value can't be
        # converted to a number, it will be replaced with NaN (Not a Number).
        df[col] = pd.to_numeric(df[col], errors='coerce')

        # Fill any missing values (NaNs) in the rank columns. We use the mean
        # (average) of the respective column to fill them. This prevents errors
        # in our calculations.
        df[col].fillna(df[col].mean(), inplace=True)

    # --- Fitness Calculation ---
    print("Calculating fitness scores...")
    # Group the entire dataset by each unique stock 'Ticker'.
    # For each ticker, calculate the mean (average) of the rank columns over the period.
    ticker_ranks = df.groupby('Ticker')[rank_columns].mean()

    # Our fitness score is the sum of the average ranks.
    # Since a lower rank is better, a lower final score indicates a "fitter" stock.
    ticker_ranks['Fitness_Score'] = ticker_ranks.sum(axis=1)

    # Sort the results by the 'Fitness_Score' in ascending order (lowest to highest).
    fittest_stocks = ticker_ranks['Fitness_Score'].sort_values(ascending=True)

    print("Calculation complete.")
    return fittest_stocks

# --- Main Execution Block ---
if __name__ == "__main__":
    # Specify the name of your dataset file.
    # Make sure this file is in the same directory as this Python script.
    dataset_file = '/content/q1_2016_all.csv'

    # Call our main function to perform the analysis.
    fitness_results = calculate_stock_fitness(dataset_file)

    # If the analysis was successful, print the results.
    if fitness_results is not None:
        print("\n--- Top 10 'Fittest' Stocks (Lower Score is Better) ---")
        # Display the top 10 stocks with the lowest (best) fitness scores.
        print(fitness_results.head(10))

Successfully loaded the dataset.
Cleaning and preparing data...
Calculating fitness scores...
Calculation complete.

--- Top 10 'Fittest' Stocks (Lower Score is Better) ---
Ticker
QED     4.409836
QGTA    4.459016
CCX     4.540984
QLS     4.590164
HYIH    4.655738
SIPE    4.754098
EFFE    4.868852
EMAG    4.901639
BICK    4.934426
ICN     4.983607
Name: Fitness_Score, dtype: float64


/tmp/ipython-input-1-1051331990.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


Step 1: Define What "Fit" Means for Your Data
First, we need to decide what makes a stock "good" or "fit" according to your dataset. Your file has these useful columns:

McapRank (Market Capitalization Rank)

TurnRank (Turnover Rank)

VolatilityRank

PriceRank

In all these cases, a lower rank is better (a rank of 1 is better than a rank of 10). Therefore, we can define a "fit" stock as one that consistently has low numbers across these four rank categories.

Step 2: Calculate the Average Performance for Each Stock
A stock's ranks can change daily. To get a stable measure of its performance over the whole quarter, we need to average them out.

Action: For each unique stock ticker (like 'AAPL', 'BAC', etc.), we calculate its average McapRank, average TurnRank, average VolatilityRank, and average PriceRank over all the days in the dataset.

This gives us one representative number for each rank category for every single stock.

Step 3: Create a Single "Fitness Score"
Now that we have four average rank numbers for each stock, we need to combine them into a single, final score so we can compare stocks directly.

Action: For each stock, we simply add its four average ranks together.

Fitness Score = (Avg McapRank) + (Avg TurnRank) + (Avg VolatilityRank) + (Avg PriceRank)

This final number is our Fitness Score. Because a lower rank is better, a lower final Fitness Score means the stock is "fitter" according to our definition.

Step 4: Compare All Stocks
Finally, we calculate this Fitness Score for every single stock in your dataset. Then, we can easily rank them from the lowest score to the highest. The stocks at the top of this list are the "fittest" because they had the best overall average ranks.

In short, that is how we apply a fitness function: by defining fitness based on the available rank data and then systematically calculating a score for each stock so they can be compared.

In [ ]:
import pandas as pd

def calculate_weighted_stock_fitness(filepath, weights):
    """
    Loads stock data and calculates a custom, weighted fitness score.

    This function allows assigning different levels of importance (weights) to
    various rank categories to create a fitness score tailored to a specific
    investment strategy. A lower score is still considered "fitter".

    Args:
        filepath (str): The path to the CSV file with the stock data.
        weights (dict): A dictionary where keys are the rank columns and
                        values are their corresponding importance (weight).
                        The sum of weights should ideally be 1.0.

    Returns:
        pandas.Series: A Series with the weighted fitness score for each stock,
                       sorted to show the fittest stocks first. Returns None on error.
    """
    try:
        df = pd.read_csv(filepath)
        print("Successfully loaded the dataset.")
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return None

    # Use the rank columns defined in the weights dictionary.
    rank_columns = list(weights.keys())

    # --- Data Cleaning and Preparation ---
    print("Cleaning and preparing data...")
    for col in rank_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].mean(), inplace=True)

    # --- Weighted Fitness Calculation ---
    print("Calculating weighted fitness scores...")
    ticker_ranks = df.groupby('Ticker')[rank_columns].mean()

    # Initialize the fitness score column to zeros.
    ticker_ranks['Weighted_Fitness_Score'] = 0.0

    # Calculate the weighted score by multiplying each average rank by its weight.
    for col, weight in weights.items():
        ticker_ranks['Weighted_Fitness_Score'] += ticker_ranks[col] * weight

    # Sort the results by the final weighted score.
    fittest_stocks = ticker_ranks['Weighted_Fitness_Score'].sort_values(ascending=True)

    print("Calculation complete.")
    return fittest_stocks

# --- Main Execution Block ---
if __name__ == "__main__":
    # --- CUSTOMIZE YOUR STRATEGY HERE ---
    # Define the importance (weight) for each rank.
    # The sum of these weights should be 1.0 (representing 100%).
    #
    # Example 1: "Conservative Investor" - Prioritizes large, stable companies.
    # We give 50% importance to McapRank and 30% to VolatilityRank.
    investor_weights = {
        'McapRank':       0.5,  # 50% importance
        'VolatilityRank': 0.3,  # 30% importance
        'TurnRank':       0.1,  # 10% importance
        'PriceRank':      0.1   # 10% importance
    }

    # Example 2: "Active Trader" - Prioritizes high turnover and price movement.
    # investor_weights = {
    #     'McapRank':       0.1,  # 10% importance
    #     'VolatilityRank': 0.2,  # 20% importance
    #     'TurnRank':       0.4,  # 40% importance
    #     'PriceRank':      0.3   # 30% importance
    # }

    # Specify the name of your dataset file.
    dataset_file = '/content/q1_2016_all.csv'

    # Call our function with the specified weights.
    fitness_results = calculate_weighted_stock_fitness(dataset_file, investor_weights)

    # If the analysis was successful, print the results.
    if fitness_results is not None:
        print(f"\n--- Top 10 'Fittest' Stocks (using '{list(investor_weights.keys())}' weights) ---")
        print(fitness_results.head(10))


Successfully loaded the dataset.
Cleaning and preparing data...
Calculating weighted fitness scores...
Calculation complete.

--- Top 10 'Fittest' Stocks (using '['McapRank', 'VolatilityRank', 'TurnRank', 'PriceRank']' weights) ---
Ticker
EFFE    1.086885
QED     1.090164
SIPE    1.091803
CCX     1.116393
QGTA    1.124590
QLS     1.163934
SCTO    1.165574
GTAA    1.170492
HYIH    1.173770
BYLD    1.190164
Name: Weighted_Fitness_Score, dtype: float64


/tmp/ipython-input-2-3744544787.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [ ]:
import pandas as pd
import os

def merge_quarterly_data(file_list, output_filename):
    """
    Merges a list of CSV files into a single DataFrame and saves it to a new CSV file.

    Args:
        file_list (list): A list of strings, where each string is the path to a CSV file.
        output_filename (str): The name for the output CSV file.

    Returns:
        bool: True if the merge was successful, False otherwise.
    """
    # Create an empty list to hold the individual DataFrames.
    dataframe_list = []

    print("Starting the merge process...")

    # Loop through each file in the provided list.
    for filename in file_list:
        # Check if the file exists before trying to read it.
        if not os.path.exists(filename):
            print(f"Error: The file '{filename}' was not found. Skipping.")
            continue

        try:
            # Read the current CSV file into a DataFrame.
            df = pd.read_csv(filename)
            # Add the DataFrame to our list.
            dataframe_list.append(df)
            print(f"Successfully loaded and added '{filename}' ({len(df)} rows).")
        except Exception as e:
            print(f"An error occurred while reading '{filename}': {e}")
            return False

    # Check if we have any DataFrames to merge.
    if not dataframe_list:
        print("No data was loaded. Aborting merge.")
        return False

    # Concatenate all the DataFrames in the list into one.
    # `ignore_index=True` re-creates the index for the new DataFrame.
    print("\nConcatenating all DataFrames...")
    merged_df = pd.concat(dataframe_list, ignore_index=True)
    print("Concatenation complete.")

    # Save the final merged DataFrame to a new CSV file.
    # `index=False` prevents pandas from writing the DataFrame index as a column.
    try:
        print(f"Saving merged data to '{output_filename}'...")
        merged_df.to_csv(output_filename, index=False)
        print("Save successful!")
        print(f"\nTotal rows in the merged file: {len(merged_df)}")
        return True
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")
        return False

# --- Main Execution Block ---
if __name__ == "__main__":
    # List of the CSV files you want to merge.
    # These files should be in the same directory as this script.
    quarterly_files = [
        '/content/q1_2023_all.csv',
        '/content/q2_2023_all.csv',
        '/content/q3_2023_all.csv',
        '/content/q4_2023_all.csv'  # <-- Corrected this line
    ]

    # The name of the new file that will contain all the merged data.
    output_file = 'full_year_2023_all.csv' # <-- Suggested a more accurate name

    # Run the merge function.
    merge_quarterly_data(quarterly_files, output_file)

Starting the merge process...
Successfully loaded and added '/content/q1_2023_all.csv' (449718 rows).
Successfully loaded and added '/content/q2_2023_all.csv' (448127 rows).
Successfully loaded and added '/content/q3_2023_all.csv' (456307 rows).
Successfully loaded and added '/content/q4_2023_all.csv' (460571 rows).

Concatenating all DataFrames...
Concatenation complete.
Saving merged data to 'full_year_2023_all.csv'...
Save successful!

Total rows in the merged file: 1814723


#With Yahoo finance

In [2]:
import pandas as pd
import yfinance as yf
import os
from datetime import datetime

def create_master_price_history(file_list, output_filename):
    """
    Creates a master price history dataset for all unique tickers found in a list of CSV files.

    This function first merges the input files to get a unique list of tickers.
    Then, it downloads the daily historical price data for each ticker for the year 2016
    from Yahoo Finance and saves it all to a single CSV file.

    Args:
        file_list (list): A list of paths to the CSV files containing trading data.
        output_filename (str): The name for the final master price history CSV file.

    Returns:
        bool: True if successful, False otherwise.
    """
    # --- Step 1: Get a unique list of all tickers ---
    all_tickers = set()
    print("Step 1: Reading your files to get a list of all unique stock tickers...")
    for filename in file_list:
        if not os.path.exists(filename):
            print(f"Warning: The file '{filename}' was not found. Skipping.")
            continue
        try:
            df = pd.read_csv(filename)
            # Add the tickers from the current file to our set.
            # A 'set' automatically handles duplicates.
            all_tickers.update(df['Ticker'].unique())
            print(f"Found {len(df['Ticker'].unique())} tickers in '{filename}'.")
        except Exception as e:
            print(f"An error occurred while reading '{filename}': {e}")
            return False

    if not all_tickers:
        print("No tickers were found. Aborting.")
        return False

    print(f"\nFound a total of {len(all_tickers)} unique tickers to download.")

    # --- Step 2: Download historical data for all tickers ---
    print("\nStep 2: Downloading 2016 price data from Yahoo Finance...")

    # We will download all data at once, which is much faster.
    # yfinance can take a space-separated string of tickers.
    ticker_string = " ".join(list(all_tickers))

    try:
        # Download daily data for the specified date range.
        # The end date is '2017-01-01' to be inclusive of the last day of 2016.
        price_data = yf.download(ticker_string, start="2016-01-01", end="2024-01-01")

        if price_data.empty:
            print("Could not download any price data. Please check the tickers.")
            return False

        # The downloaded data has multi-level columns. We need to reformat it.
        # We'll stack the data to turn the tickers from columns into a row value.
        price_data = price_data.stack().reset_index()
        price_data.rename(columns={'level_1': 'Ticker'}, inplace=True)

        print("Successfully downloaded all price data.")

    except Exception as e:
        print(f"An error occurred during download: {e}")
        return False

    # --- Step 3: Save the merged data to a new CSV file ---
    try:
        print(f"\nStep 3: Saving the master price history to '{output_filename}'...")
        price_data.to_csv(output_filename, index=False)
        print("Save successful!")
        print(f"The file '{output_filename}' has been created with {len(price_data)} rows.")
        return True
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")
        return False

# --- Main Execution Block ---
if __name__ == "__main__":
    # Before running, you need to install the yfinance library.
    # Open your terminal or command prompt and type:
    # pip install yfinance

    # List of your quarterly data files.
    quarterly_files = [
       '/content/training_activity_data_2016-2023.csv'
    ]

    # The name of the new file we want to create.
    output_price_file = 'master_price_history_2016-2023.csv'

    # Run the main function.
    create_master_price_history(quarterly_files, output_price_file)

Step 1: Reading your files to get a list of all unique stock tickers...
Found 11223 tickers in '/content/training_activity_data_2016-2023.csv'.

Found a total of 11223 unique tickers to download.

Step 2: Downloading 2016 price data from Yahoo Finance...


/tmp/ipython-input-2-2402427726.py:54: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.download(ticker_string, start="2016-01-01", end="2024-01-01")
[*********************100%***********************]  11223 of 11223 completed
ERROR:yfinance:
11221 Failed downloads:
ERROR:yfinance:['WIFI', 'CLOV', 'OCA', 'PGF', 'CUE', 'RGLB', 'CRDT', 'CHRD', 'NCBS', 'GVT', 'DFLI', 'IBMQ', 'TSLI', 'AEGN', 'DIET', 'WNC', 'NN', 'PFFD', 'SPFF', 'DTOX', 'FDMT', 'SMK', 'QRFT', 'OBNK', 'PERF', 'TLDH', 'CPN', 'BILZ', 'ATXI', 'IT', 'PNX', 'REPL', 'RTYD', 'OBIL', 'CPSS', 'GLEE', 'MEET', 'DAT', 'NUSA', 'PHIIK', 'ICOP', 'GSEV', 'CFI', 'IFV', 'WEST', 'BSJL', 'TSLS', 'FINQ', 'CGIE', 'FCVA', 'VWTR', 'MYE', 'EMCI', 'FISI', 'AFRM', 'IBIA', 'ANGI', 'TAFI', 'NFBK', 'MSP', 'MUST', 'SEB', 'RLY', 'GPOR', 'CIA', 'ESIX', 'EDUT', 'IAT', 'PRVT', 'TCAF', 'TYPE', 'NVDX', 'ADIV', 'ENFN', 'VCAR', 'CF', 'HCII', 'PPA', 'URA', 'FFIC', 'TTSH', 'DNOW', 'KEG', 'RTNB', 'EWJV', 'PYPL', 'LSEA',

Successfully downloaded all price data.

Step 3: Saving the master price history to 'master_price_history_2016-2023.csv'...
Save successful!
The file 'master_price_history_2016-2023.csv' has been created with 2428 rows.


In [ ]:
import pandas as pd
import os

def merge_yearly_data_efficiently(file_list, output_filename):
    """
    Merges a list of large yearly CSV files into a single file in a memory-efficient way.

    It writes the first file with a header, then appends the rest of the files
    without headers, avoiding loading all data into memory at once.

    Args:
        file_list (list): A list of paths to the yearly data CSV files.
        output_filename (str): The name for the final, merged output CSV file.

    Returns:
        bool: True if the merge was successful, False otherwise.
    """
    if not file_list:
        print("Error: The file list is empty.")
        return False

    print("Starting the memory-efficient merge process...")

    # --- Step 1: Write the first file with its header ---
    first_file = file_list[0]
    if not os.path.exists(first_file):
        print(f"Error: The first file '{first_file}' was not found. Aborting.")
        return False

    try:
        print(f"Processing '{first_file}' as the base file...")
        # Write the first file to the output file in 'write' mode ('w')
        df_initial = pd.read_csv(first_file)
        df_initial.to_csv(output_filename, index=False, mode='w')
        total_rows = len(df_initial)
        print(f"Successfully wrote {total_rows} rows from the first file.")
    except Exception as e:
        print(f"An error occurred with the first file '{first_file}': {e}")
        return False

    # --- Step 2: Append the remaining files without their headers ---
    for filename in file_list[1:]:
        if not os.path.exists(filename):
            print(f"Warning: The file '{filename}' was not found. Skipping.")
            continue

        try:
            print(f"Appending data from '{filename}'...")
            df_chunk = pd.read_csv(filename)
            # Append to the CSV using mode='a' (append) and without the header
            df_chunk.to_csv(output_filename, index=False, mode='a', header=False)
            total_rows += len(df_chunk)
            print(f"Appended {len(df_chunk)} rows. Total rows so far: {total_rows}")
        except Exception as e:
            print(f"An error occurred while appending '{filename}': {e}")
            return False

    print("\nMerge complete!")
    print(f"The merged file '{output_filename}' has been created with {total_rows} rows.")
    return True

# --- Main Execution Block ---
if __name__ == "__main__":
    # 1. List of the yearly CSV files you want to merge.
    #    Make sure these files are in the correct location.
    yearly_files = [
        '/content/full_year_2016_all.csv',
        '/content/full_year_2017_all.csv',
        '/content/full_year_2018_all.csv',
        '/content/full_year_2019_all.csv',
        '/content/full_year_2020_all.csv',
        '/content/full_year_2021_all.csv',
        '/content/full_year_2022_all.csv',
        '/content/full_year_2023_all.csv'
    ]

    # 2. The name of the new file that will contain all the merged data.
    output_file = 'training_activity_data_2016-2023.csv'

    # 3. Run the memory-efficient merge function.
    merge_yearly_data_efficiently(yearly_files, output_file)

Starting the memory-efficient merge process...
Processing '/content/full_year_2016_all.csv' as the base file...
Successfully wrote 1331611 rows from the first file.
Appending data from '/content/full_year_2017_all.csv'...
Appended 1314113 rows. Total rows so far: 2645724
Appending data from '/content/full_year_2018_all.csv'...
Appended 1373225 rows. Total rows so far: 4018949
Appending data from '/content/full_year_2019_all.csv'...
Appended 1442504 rows. Total rows so far: 5461453
Appending data from '/content/full_year_2020_all.csv'...
Appended 1470397 rows. Total rows so far: 6931850
Appending data from '/content/full_year_2021_all.csv'...
Appended 1649533 rows. Total rows so far: 8581383
Appending data from '/content/full_year_2022_all.csv'...
Appended 1818583 rows. Total rows so far: 10399966
Appending data from '/content/full_year_2023_all.csv'...
Appended 1814723 rows. Total rows so far: 12214689

Merge complete!
The merged file 'training_activity_data_2016-2023.csv' has been cre

In [ ]:
df = pd.read_csv('/content/training_activity_data_2016-2023.csv')
df.tail()


,Date,Security,Ticker,McapRank,TurnRank,VolatilityRank,PriceRank,LitVol('000),OrderVol('000),Hidden,TradesForHidden,HiddenVol('000),TradeVolForHidden('000),Cancels,LitTrades,OddLots,TradesForOddLots,OddLotVol('000),TradeVolForOddLots('000)
12214684,20231229.0,ETF,ZSB,1.0,2.0,2.0,1.0,0.000,666.500,1.0,1.0,0.017,0.017,4058.0,0.0,1.0,1.0,0.017,0.017
12214685,20231229.0,ETF,ZSC,1.0,1.0,4.0,2.0,0.000,633.600,1.0,1.0,0.001,0.001,2829.0,0.0,1.0,1.0,0.001,0.001
12214686,20231229.0,ETF,ZSL,2.0,4.0,4.0,1.0,286.939,233627.851,144.0,1936.0,18.635,305.574,321438.0,1788.0,306.0,1931.0,9.770,305.058
12214687,20231229.0,ETF,ZTAX,1.0,1.0,1.0,2.0,0.000,21.600,0.0,0.0,0.000,0.000,218.0,0.0,0.0,0.0,0.000,0.000
12214688,20231229.0,ETF,ZZZ,1.0,2.0,3.0,1.0,0.000,558.400,1.0,1.0,0.001,0.001,2422.0,0.0,1.0,1.0,0.001,0.001


In [ ]:
df.head()

,Date,Security,Ticker,McapRank,TurnRank,VolatilityRank,PriceRank,LitVol('000),OrderVol('000),Hidden,TradesForHidden,HiddenVol('000),TradeVolForHidden('000),Cancels,LitTrades,OddLots,TradesForOddLots,OddLotVol('000),TradeVolForOddLots('000)
0,20160104.0,Stock,A,10.0,7.0,1.0,8.0,2146.907,105397.221,1552.0,19804.0,260.172,2407.079,312569.0,14066.0,3774.0,15289.0,159.058,1639.151
1,20160104.0,Stock,AA,10.0,10.0,5.0,4.0,17724.383,560920.682,7057.0,57895.0,2123.911,19848.294,563489.0,46314.0,4940.0,52454.0,204.582,13065.252
2,20160104.0,Stock,AAC,5.0,5.0,6.0,6.0,61.810,5108.669,74.0,779.0,7.522,69.332,20662.0,572.0,214.0,633.0,8.835,56.071
3,20160104.0,Stock,AAL,10.0,10.0,3.0,8.0,6661.915,138785.390,5736.0,63879.0,920.243,7581.529,813998.0,55105.0,12007.0,60807.0,525.661,7212.508
4,20160104.0,Stock,AAMC,2.0,10.0,10.0,6.0,28.275,1983.899,117.0,422.0,10.659,38.934,7294.0,260.0,105.0,367.0,3.416,31.979
